In [ ]:
from tensorflow import keras

inception_model = keras.models.load_model("inception_model.keras")

2025-04-04 13:23:17.822641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-04 13:23:17.822673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-04 13:23:17.823602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 13:23:17.829071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-04 13:23:18.513143: W tensorflow/compiler/tf2

In [ ]:
from transformers import ViTModel, ViTFeatureExtractor

vit_model = ViTModel.from_pretrained("vit_model")
feature_extractor = ViTFeatureExtractor.from_pretrained("feature_extractor")

/home/aubct/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aubct/.local/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
from tensorflow import keras

classifier = keras.models.load_model("classifier.keras")

In [ ]:
import joblib

label_encoder = joblib.load("label_encoder.pkl")

In [ ]:
import cv2
import numpy as np
import torch

def preprocess_image(image_path, size=(299, 299)):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, size)
    img_denoised = cv2.bilateralFilter(img_resized, d=11, sigmaColor=75, sigmaSpace=60)
    img_gray = cv2.cvtColor(img_denoised, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_gray)

    edges = cv2.Canny(img_clahe, 100, 200)
    img_combined = cv2.addWeighted(img_clahe, 0.7, edges, 0.3, 0)

    img_rgb = np.stack([img_combined] * 3, axis=-1)
    img_rgb = img_rgb.astype(np.float32) / 255.0
    return np.expand_dims(img_rgb, axis=0)

def extract_inception_features(img):
    return inception_model.predict(img)

def extract_vit_features(img):
    inputs = feature_extractor(img, return_tensors="pt")
    with torch.no_grad():
        outputs = vit_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

def predict_new_image(image_path):
    img = preprocess_image(image_path)

    inception_features = extract_inception_features(img)
    vit_features = extract_vit_features(img)

    w_inception, w_vit = 0.7, 0.3
    fused_features = np.concatenate([w_inception * inception_features, w_vit * vit_features], axis=1)

    prediction = classifier.predict(fused_features)
    print("Prediction probs:", prediction)
    predicted_class = np.argmax(prediction)

    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_label


In [ ]:
image_path1 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/bacterial_leaf_blight/bacterial_leaf_blight (16).JPG"

predicted_disease = predict_new_image(image_path1)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 12ms/step
Prediction probs: [[9.7302771e-01 7.6339693e-06 2.4770367e-05 4.8421123e-05 1.2632051e-02
  2.0050541e-03 4.6770197e-07 2.9264067e-03 6.9793041e-06 9.3204109e-03]]
Predicted Disease: bacterial_leaf_blight


In [ ]:
image_path2 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/brown_spot/brown_spot (115).jpg"

predicted_disease = predict_new_image(image_path2)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: brown_spot


In [ ]:
image_path3 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/healthy/healthy (233).jpg"

predicted_disease = predict_new_image(image_path3)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: healthy


In [ ]:
image_path4 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/leaf_blast/leaf_blast (54).jpg"

predicted_disease = predict_new_image(image_path4)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 12ms/step
Predicted Disease: leaf_blast


In [ ]:
image_path5 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/leaf_scald/leaf_scald (17).jpg"

predicted_disease = predict_new_image(image_path5)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: leaf_scald


In [ ]:
image_path6 = "/home/aubct/Downloads/archive(1)/RiceLeafsDisease/train/narrow_brown_spot/narrow_brown (5).jpg"

predicted_disease = predict_new_image(image_path6)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: narrow_brown_spot


In [ ]:
image_path7 = "/home/aubct/Downloads/archive/Rice_Leaf_Diease/Rice_Leaf_Diease/test/Rice Hispa/Rice_Hispa (21).jpg"

predicted_disease = predict_new_image(image_path7)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: rice_hispa


In [ ]:
image_path8 = "/home/aubct/Downloads/archive/Rice_Leaf_Diease/Rice_Leaf_Diease/test/Sheath Blight/IMG_20231014_171742.jpg"

predicted_disease = predict_new_image(image_path8)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: sheath_blight


In [ ]:
image_path9 = "/home/aubct/Downloads/archive/Rice_Leaf_Diease/Rice_Leaf_Diease/test/Tungro/IMG_0897.jpg"

predicted_disease = predict_new_image(image_path9)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: tungro


In [ ]:
image_path10 = "/home/aubct/Downloads/archive/Rice_Leaf_Diease/Rice_Leaf_Diease/test/Neck_Blast/IMG20201109210551_00.jpg"

predicted_disease = predict_new_image(image_path10)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 13ms/step
Predicted Disease: neck_blast
